# Serper.dev와 LangChain, CrewAI, Tools, Agents를 연결한 웹 정보 검색, 요약, 한국어 번역
- Serper.dev와 LangChain, CrewAI, Tools, Agents를 모두 연결한 "웹 정보 검색 → 요약 → 한국어 번역" 자동화 시스템을 구현하는 예제이다.

## 프로젝트 목표 
- 영어로 주어진 주제
- serper.dev로 검색
- GPT로 요약
- 한국어로 번역
- 위 전체 프로세스를 CrewAI 멀티에이전트로 자동화

## 전체 구조
1. WebSearchAgent → Serper.dev로 검색
2. SummaryAgent → 영어 요약
3. TranslateAgent → 한국어 번역  
→ 모두 CrewAI 기반 Agent로 구성


In [7]:
# 필요 패키지 설치 (Jupyter 또는 터미널)
# !pip install crewai==0.20.11 crewai-tools==0.48.0
!pip install openai langchain langchain-openai crewai crewai-tools

  Using cached litellm-1.72.0-py3-none-any.whl.metadata (39 kB)
Using cached litellm-1.72.0-py3-none-any.whl (8.0 MB)
  Attempting uninstall: litellm
    Found existing installation: litellm 1.68.0
    Uninstalling litellm-1.68.0:
      Successfully uninstalled litellm-1.68.0
  Attempting uninstall: crewai
    Found existing installation: crewai 0.120.0
    Uninstalling crewai-0.120.0:
      Successfully uninstalled crewai-0.120.0


In [9]:
!pip show crewai
!pip show crewai-tools
!python --version

Name: crewai
Version: 0.134.0
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: https://crewai.com
Author: 
Author-email: Joao Moura <joao@crewai.com>
License: 
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: appdirs, auth0-python, blinker, chromadb, click, instructor, json-repair, json5, jsonref, litellm, onnxruntime, openai, openpyxl, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pdfplumber, pydantic, python-dotenv, pyvis, regex, tokenizers, tomli, tomli-w, uv
Required-by: crewai-tools
Name: crewai-tools
Version: 0.47.1
Summary: Set of tools for the crewAI framework
Home-page: https://crewai.com
Author: 
Author-email: João Moura <joaomdmoura@gmail.com>
License: 
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: chromadb, click, crewai, docker, embedchain, lanced

In [13]:
import os
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from langchain_openai import ChatOpenAI
from openai import OpenAI

# ✅ API 키 설정
os.environ["SERPER_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

# ✅ LLM 초기화
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.3)

# ✅ 검색 도구
search_tool = SerperDevTool()

# ✅ 1. 웹 검색 Agent
web_researcher = Agent(
    role="Web Researcher",
    goal="Search for English information about a given topic",
    backstory="An expert at gathering recent and relevant information using search engines.",
    tools=[search_tool],
    llm=llm,
    verbose=True,
)

# ✅ 2. 영어 요약 Agent
summarizer = Agent(
    role="Summarizer",
    goal="Summarize English web content into concise and clear language",
    backstory="A professional content summarizer for technical and general topics.",
    llm=llm,
    verbose=True,
)

# ✅ 3. 한국어 번역 Agent
translator = Agent(
    role="Translator",
    goal="Translate English summaries into fluent Korean",
    backstory="A bilingual language expert skilled in translating English to Korean accurately and naturally.",
    llm=llm,
    verbose=True,
)

# ✅ 사용자 주제
topic = "Latest applications of AI in education"

# ✅ Task 1: 웹에서 정보 검색
task1 = Task(
    description=f"Search Google for 3-5 recent English articles or summaries about: {topic}",
    expected_output="A list of 3-5 relevant English snippets or paragraphs about the topic",
    agent=web_researcher,
)

# ✅ Task 2: 검색 결과 요약
task2 = Task(
    description="Summarize the English content into 1-2 paragraphs with clear language.",
    expected_output="A clean and concise English summary.",
    agent=summarizer,
    # depends_on=[task1]
)

# ✅ Task 3: 한국어로 번역
task3 = Task(
    description="Translate the English summary into fluent Korean.",
    expected_output="A well-written Korean version of the English summary.",
    agent=translator,
    depends_on=[task2]
)

# ✅ Crew 설정 및 실행
crew = Crew(
    agents=[web_researcher, summarizer, translator],
    tasks=[task1, task2, task3],
    verbose=True
)

# search_tool.run("test") 
# ✅ Crew 실행 전에 각 Task 상태 확인 (디버깅용)
print(">> 웹 검색 시작")
print(task1)

print(">> 요약 작업 시작")
print(task2)

print(">> 번역 작업 시작")
print(task3)

result = crew.kickoff()
print("📝 최종 결과 (한국어 요약):\n")
print(result)


>> 웹 검색 시작
used_tools=0 tools_errors=0 delegations=0 i18n=I18N(prompt_file=None) name=None prompt_context=None description='Search Google for 3-5 recent English articles or summaries about: Latest applications of AI in education' expected_output='A list of 3-5 relevant English snippets or paragraphs about the topic' config=None callback=None agent=Agent(role=Web Researcher, goal=Search for English information about a given topic, backstory=An expert at gathering recent and relevant information using search engines.) context=NOT_SPECIFIED async_execution=False output_json=None output_pydantic=None output_file=None output=None tools=[SerperDevTool(name='Search the internet with Serper', description="Tool Name: Search the internet with Serper\nTool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}\nTool Description: A tool that can be used to search the internet with a search_query. Supports different search types:

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f434f414-50dd-447c-b8f2-fee4bf8d01ff                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Researcher                                                                                          │
│                                                                                                                 │
│  Task: Search Google for 3-5 recent English articles or summaries about: Latest applications of AI in           │
│  education                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/opt/anaconda3/lib/python3.12/site-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Translator                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  교육 분야에서의 다양한 업무 관리, 성적 평가, 수업 계획, 그리고 소통 등을 처리하면서 교육자들은 인공지능(AI)의  │
│  교육 분야에서의 미래에 대해 혼합된 감정을 표현하고 있습니다. 일부는 AI를 통해 행정 업무에 소요되는 시간을      │
│  절약하고 학생 참여를 증진시킬 수 있는 유용한 도구로 보지만, 다른 사람들은 최근 대통령이 AI를 K-12 교육에서     │
│  촉진하기 위해 4월 23일에 행정명령을 발표함으로써 강조된 것처럼, 핵심 사고 능력에 미치는 부정적인 영향에 대한   │
│  우려를 표현하고 있습니다. 현재 교육 분야에서의 AI 적용은 적응형 지도 시스템, 자동 쓰기 평가 시스템, 챗봇 등    │
│  학생을 지원하는 도구를 포함하며, 맞춤형 학습, 지능형 지도 시스템, 자동 채점 및 피드백 시스템과 같은 요소도     │
│  함께 진행되고 있습니다. 현대 교육 분야에서의 AI 역할은 행정 업무를 간소화하고 맞춤형 학습 경험을 제공함으로써  │
│  발전하고 있으며, 교육자들 사이에서 AI 기술이 교육 분야에서 미래에 미치는 영향에 대한 낙관론과 우려가 공존하고  │
│  있습니다.                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 177fab36-5313-4989-bdcf-7ee352735216                                                                     │
│  Agent: Translator                                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f434f414-50dd-447c-b8f2-fee4bf8d01ff                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 교육 분야에서의 다양한 업무 관리, 성적 평가, 수업 계획, 그리고 소통 등을 처리하면서 교육자들은   │
│  인공지능(AI)의 교육 분야에서의 미래에 대해 혼합된 감정을 표현하고 있습니다. 일부는 AI를 통해 행정 업무에       │
│  소요되는 시간을 절약하고 학생 참여를 증진시킬 수 있는 유용한 도구로 보지만, 다른 사람들은 최근 대통령이 AI를   │
│  K-12 교육에서 촉진하기 위해 4월 23일에 행정명령을 발표함으로써 강조된 것처럼, 핵심 사고 능력에 미치는          │
│  부정적인 영향에 대한 우려를 표현하고 있습니다. 현재 교육 분야에서의 AI 적용은 적응형 지도 시스템, 자동 쓰기    │
│  평가 시스템, 챗봇 등 학생을 지원하는 도구를 포함하며, 맞춤형 학습, 지능형 지도 시스템, 자동 채점 및 피드백     │
│  시스템과 같은 요소도 함께 진행되고 있습니다. 현대 교육 분야에서의 AI 역할은 행정 업무를 간소화하고 맞춤형      │
│  학습 경험을 제공함으로써 발전하고 있으며, 교육자들 사이에서 AI 기술이 교육 분야에서 미래에 미치는 영향에 대한  │
│  낙관론과 우려가 공존하고 있습니다.                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

📝 최종 결과 (한국어 요약):

교육 분야에서의 다양한 업무 관리, 성적 평가, 수업 계획, 그리고 소통 등을 처리하면서 교육자들은 인공지능(AI)의 교육 분야에서의 미래에 대해 혼합된 감정을 표현하고 있습니다. 일부는 AI를 통해 행정 업무에 소요되는 시간을 절약하고 학생 참여를 증진시킬 수 있는 유용한 도구로 보지만, 다른 사람들은 최근 대통령이 AI를 K-12 교육에서 촉진하기 위해 4월 23일에 행정명령을 발표함으로써 강조된 것처럼, 핵심 사고 능력에 미치는 부정적인 영향에 대한 우려를 표현하고 있습니다. 현재 교육 분야에서의 AI 적용은 적응형 지도 시스템, 자동 쓰기 평가 시스템, 챗봇 등 학생을 지원하는 도구를 포함하며, 맞춤형 학습, 지능형 지도 시스템, 자동 채점 및 피드백 시스템과 같은 요소도 함께 진행되고 있습니다. 현대 교육 분야에서의 AI 역할은 행정 업무를 간소화하고 맞춤형 학습 경험을 제공함으로써 발전하고 있으며, 교육자들 사이에서 AI 기술이 교육 분야에서 미래에 미치는 영향에 대한 낙관론과 우려가 공존하고 있습니다.


CrewAI 기반 영어 정보 수집 및 한국어 번역 자동화 프로젝트 보고서

1. 프로젝트 개요

이 프로젝트는 CrewAI 플랫폼을 활용하여 다중 에이전트 시스템을 구성하고, 웹에서 영어 정보를 검색하고 이를 요약 및 한국어로 번역하는 자동화된 정보 처리 파이프라인을 구축하는 것을 목표로 한다. 특히, 검색 도구로는 SerperDevTool, 요약 및 번역 도구로는 OpenAI 기반 LLM(ChatOpenAI)을 활용하였다.

2. 사용 기술 및 구성 요소

2.1 사용 기술

Python 3.12

CrewAI (crewai==0.20.11)

crewai-tools (crewai-tools==0.48.0)

LangChain (langchain-openai)

OpenAI GPT (gpt-3.5-turbo)

Serper.dev API (Google Search proxy)

2.2 주요 도구

SerperDevTool: 웹 검색 수행 (검색어 기반)

ChatOpenAI: 검색 결과 요약 및 번역에 사용되는 LLM

3. 시스템 아키텍처 및 역할

본 시스템은 다음과 같은 3개의 Agent와 3개의 Task로 구성된다.

3.1 에이전트

Web Researcher: SerperDevTool을 사용하여 주제에 대한 영어 정보를 검색

Summarizer: 수집된 정보를 명확한 영어 요약으로 정리

Translator: 영어 요약을 자연스러운 한국어로 번역

3.2 태스크

Task 1: 사용자가 입력한 주제에 대해 3~5개의 영어 기사/요약 검색

Task 2: 수집된 영어 정보 요약

Task 3: 요약된 영어를 한국어로 번역

4. 실행 코드 요약

# 주요 에이전트 및 태스크 정의
crew = Crew(
    agents=[web_researcher, summarizer, translator],
    tasks=[task1, task2, task3],
    verbose=True
)
result = crew.kickoff()
print(result)

API Key는 os.environ을 통해 환경 변수로 설정

실행 결과는 한국어 번역된 요약으로 반환

5. 결과 예시

입력 주제: Latest applications of AI in education

요약된 영어 정보:

AI is being applied to personalized learning platforms, automated grading systems, and smart tutoring tools...

번역된 한국어 요약:

인공지능은 개인화된 학습 플랫폼, 자동 채점 시스템, 스마트 튜터링 도구 등에 활용되고 있습니다...

6. 결론 및 확장 가능성

이 프로젝트는 CrewAI의 멀티에이전트 구조를 활용하여 검색-요약-번역이라는 작업 흐름을 자동화할 수 있음을 보여주었다. 이를 확장하면:

다국어 번역 시스템 구축

뉴스/학술정보 실시간 분석

음성 및 이미지 요약 등으로 확장 가능하다.

7. 참고 자료

https://docs.crewai.io

https://serper.dev

https://platform.openai.com

